In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#from IPython.display import display, display_latex, Math, Latex
from constants import CGS, MKS

## D-D Fusion.

First some basic quantities:

* Assume D2O fuel, DD fusion with efficiency $\eta$ at turning fusion power into beam power.  Beam power is used to accelerate the O ions in an ion rocket.
* DD fusion energy = $\rm E_{DD}$ = 10 MeV per DD pair.
* This energy is used to accelerate the O at a velocity $\rm \frac{m_O v_O^2}{2} = E_{DD}$
* $\rm v_O = \sqrt{2 \eta E_{DD} / m_O}$
* Effective velocity $\rm V_{eff}$ is reduced by $\rm \frac{m_O}{m_{D_2O}}$
* So: $\rm V_{eff} = \frac{m_O}{m_{D_2O}}\sqrt{2 \eta E_{DD} / m_O}$

In [ ]:
eta = 0.20
mDD = 4.0; mD2O = 20.0; mO = 16.0
EDD = 1.0E7 * MKS.eV
Veff = mO/mD2O * np.sqrt(2 * eta * EDD / (mO * MKS.mp))
print("Veff = %.4g m/s = %.1f%% of c"%(Veff,Veff/MKS.c*100.0))

## Mass definitions

Let's define the following quantities:
* m0 - Mass of craft with no fuel
* mt - turnover mass
* mi - initial mass - craft plus fuel
* D = total distance; T = total trip time

  Now let's calculate these quantities, assuming the rocket equation:
$\rm v_f = v_{eff} \log{\frac{m_i}{m_f}}$
* Since the velocity at turnover is the same accelerating and declerating, we must have:
* $\rm \frac{m_t}{m_0} = \frac{m_i}{m_t}$
* $\rm m_t = \sqrt{m_0 m_i} = \sqrt{m_0 (m_0 + mdot T)}$

In [ ]:
def Velocity(Veff, mf, mi, mdot):
    # From the rocket equation
    return Veff * np.log(mi / mf)

def Distance(Veff, mf, mi, mdot):
    # From integrating the rocket equation
    t = abs((mi - mf) / mdot)
    x1 = Veff * t
    x2 = Veff * (mi / mdot) * (1.0 - mdot * t / mi) * np.log(1.0 - mdot * t / mi)
    return (x1 + x2) / MKS.ly

def TotalDistance(Veff, m0, mdot, T):
    # Total distance traveled given m0, mdot and time T
    mi = m0 + mdot * T *MKS.yr
    mt = np.sqrt(m0 * (m0 + mdot * T * MKS.yr))
    Tt = (mi - mt) / mdot / MKS.yr
    Vt = Velocity(Veff, mt, mi, mdot) / MKS.c # V at turnover
    d_accel = Distance(Veff, mt, mi, mdot)
    d_decel = Vt * (T - Tt) - Distance(Veff, m0, mt, mdot)
    return d_accel + d_decel

def TotalTime(Veff, m0, mdot, D, Tguess):
    # Invert TotalDistance(T) using Newton's method
    # Gives total time to travel a distance D given m0 and mdot
    d = TotalDistance(Veff, m0, mdot, Tguess)
    T = Tguess
    error = D - d
    count = 0
    while error > 1.0E-6 and count < 20:
        d = TotalDistance(Veff, m0, mdot, T)
        dp = TotalDistance(Veff, m0, mdot, T * 1.01)
        dDdT = (dp - d) / (T * .01)
        T = T - (d - D) / dDdT
        d = TotalDistance(Veff, m0, mdot, T)
        error = abs(D - d)
        count += 1
    if count > 15:
        print("Failed to converge")
        return 0.0
    else:
        return T


In [ ]:
m0 = 2.0E8
mdot = 0.10
D = 4.3
T = TotalTime(Veff, m0, mdot, D, 100)
print("Total time = %.1f years"%T)
mi = m0 + mdot * TotalTime(Veff, m0, mdot, D, 100) * MKS.yr
print("Initial mass = %.2g kg = %.1f times m0"%(mi,mi/m0))
mt = np.sqrt(m0 * (m0 + T * mdot * MKS.yr))
print("Turnover mass = %.2g kg = %.1f times m0"%(mt,mt/m0))
Tt = (mi - mt) / mdot / MKS.yr
print("Turnover time = %.1f years"%Tt)
Vp = Velocity(Veff, mt, mi, mdot) / MKS.c * 100.0
print("Peak velocity = %.1f%% of c"%Vp)
Thrust = Veff * mdot
Ai = Thrust / mi / 9.8
Af = Thrust / m0 / 9.8
print("Thrust = %.3g Nt"%Thrust)
print("Initial acceleration = %.3g g, Final acceleration = %.3g g"%(Ai,Af))

In [ ]:
nsteps = 500
times = np.linspace(0, T, nsteps)
dt = T / float(nsteps - 1)
speeds = []
distances = [0.0]
Vt = Velocity(Veff, mt, mi, mdot) / MKS.c # At turnover
Dt = Distance(Veff, mt, mi, mdot) # At turnover
for t in times:
    m = mi - mdot * t * MKS.yr
    if t < Tt:
        speeds.append(Velocity(Veff, m, mi, mdot) / MKS.c)
        distances.append(Distance(Veff, m, mi, mdot))
    else:
        speeds.append(Vt - Velocity(Veff, m, mt, mdot) / MKS.c)
        distances.append(Dt + Vt * (t - Tt) - Distance(Veff, m, mt, mdot))
distances.remove(distances[-1])
plt.figure(figsize = (16,8))
plt.subplot(1,2,1)
plt.title("Speed vs time")
plt.plot(times, speeds)
plt.ylim(0, 0.020)
plt.xlabel("Time(yr)")
plt.ylabel("Speed (fraction of c)")
plt.text(Tt*0.95, Vp/100.0*1.05, "Turnover", fontsize=12)
plt.subplot(1,2,2)
plt.title("Distance vs time")
plt.plot(times, distances)
plt.ylim(0, 5.0)
plt.xlabel("Time(yr)")
plt.ylabel("Distance (ly)")
plt.show()

In [ ]:
nsteps = 100
mdots = np.linspace(0, 0.5, nsteps)
mdots = np.delete(mdots, mdots[0])
powers = []
times = []
D = 4.3
for mdot in mdots:
    powers.append(eta * EDD / (mD2O * MKS.mp) * mdot * 1E-12)
    times.append(TotalTime(Veff, m0, mdot, D, 100))
plt.figure(figsize = (16,8))
plt.subplot(1,2,1)
plt.title("Electrical power needed vs mdot")
plt.plot(mdots, powers)
plt.plot([0.1,0.1], [0.0,5.0], ls = '--', color = 'k')
plt.ylim(0, 5.0)
plt.xlabel("mdot(kg/sec)")
plt.ylabel("Power (TW)")
plt.subplot(1,2,2)
plt.title("Trip time vs mdot")
plt.plot(mdots, times)
plt.plot([0.1,0.1], [0.0,1500], ls = '--', color = 'k')
plt.ylim(0, 1500.0)
plt.xlabel("mdot(kg/sec)")
plt.ylabel("Trip Time (years)")


plt.show()

In [ ]:
# What about going further?
m0 = 2.0E8
mdot = 0.10
D = 20.0
T = TotalTime(Veff, m0, mdot, D, 100)
print("Total time = %.1f years"%T)
mi = m0 + mdot * TotalTime(Veff, m0, mdot, D, 100) * MKS.yr
print("Initial mass = %.2g kg = %.1f times m0"%(mi,mi/m0))
mt = np.sqrt(m0 * (m0 + T * mdot * MKS.yr))
print("Turnover mass = %.2g kg = %.1f times m0"%(mt,mt/m0))
Tt = (mi - mt) / mdot / MKS.yr
print("Turnover time = %.1f years"%Tt)
Vp = Velocity(Veff, mt, mi, mdot) / MKS.c * 100.0
print("Peak velocity = %.1f%% of c"%Vp)